<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 9.6: Sentiment Analysis
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

Based on the video tutorial **Text Classification with Machine Learning,SpaCy and Scikit(Sentiment Analysis)** by **Jesse E. Agbe (JCharis)**.

## Data Source: UCI
### UCI - Machine Learning Repository
- Center for Machine Learning and Intelligent Systems

The [**UCI Machine Learning Repository**](http://archive.ics.uci.edu/ml/about.html) is a collection of databases, domain theories, and data generators that are used by the machine learning community for the empirical analysis of machine learning algorithms.

### Dataset
- [Sentiment Labelled Sentences Data Set](http://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences)

### Abstract
The dataset contains sentences labelled with positive or negative sentiment.

- Data Set Characteristics: Text
- Number of Instances: 3000
- Area: N/A
- Attribute Characteristics: N/A
- Number of Attributes: N/A
- Date Donated: 2015-05-30
- Associated Tasks: Classification
- Missing Values? N/A
- Number of Web Hits: 102584

### Source
Dimitrios Kotzias dkotzias '@' ics.uci.edu

### Data Set Information
This dataset was created for the Paper 'From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015

Please cite the paper if you want to use it :)

It contains sentences labelled with positive or negative sentiment.

### Format
sentence &lt;tab&gt; score &lt;newline&gt;

### Details
Score is either 1 (for positive) or 0 (for negative)

The sentences come from three different websites/fields:
- imdb.com
- amazon.com
- yelp.com

For each website, there exist **500 positive** and **500 negative** sentences. Those were selected randomly for larger datasets of reviews.

We attempted to select sentences that have a clearly positive or negative connotaton, the goal was for no neutral sentences to be selected.

For the full datasets look:

- **imdb**: Maas et. al., 2011 _Learning word vectors for sentiment analysis_
- **amazon**: McAuley et. al., 2013 _Hidden factors and hidden topics: Understanding rating dimensions with review text_
- **yelp**: [Yelp dataset challenge](http://www.yelp.com/dataset_challenge)


### Attribute Information
The attributes are text sentences, extracted from reviews of products, movies, and restaurants

### Relevant Papers
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

### Citation Request
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

## Import libraries

In [1]:
## Import Libraries
import pandas as pd

import regex as re
import spacy

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import warnings
warnings.filterwarnings('ignore')

## Load data

Load Yelp, Amazon and Imdb Data.

Hint: Source is separated by **tabs** and has no headers.

In [2]:
yelp_text = 'dat\yelp_labelled.txt'
imdb_text = 'dat\imdb_labelled_fixed.txt'
amazon_text = 'dat\\amazon_cells_labelled.txt'


In [3]:
yelp = pd.read_csv(yelp_text, sep='\t', names=['text', 'label'])
imdb = pd.read_csv(imdb_text, sep='\t', names=['text', 'label'])
amazon = pd.read_csv(amazon_text, sep='\t', names=['text', 'label'])

In [4]:
yelp

,text,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [5]:
imdb

,text,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
995,I just got bored watching Jessice Lange take h...,0
996,"Unfortunately, any virtue in this film's produ...",0
997,"In a word, it is embarrassing.",0
998,Exceptionally bad!,0


In [6]:
amazon

,text,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


## Merge the data

Merge all three datasets.

In [7]:
# ANSWER
df = [yelp, imdb, amazon]
data_text = pd.concat(df, ignore_index=True)
data_text

,text,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
2995,The screen does get smudged easily because it ...,0
2996,What a piece of junk.. I lose more calls on th...,0
2997,Item Does Not Match Picture.,0
2998,The only thing that disappoint me is the infra...,0


## Export the data

Export merged datasets to as csv file.

In [8]:
data_text.to_csv('dat\merged_reviews.csv', index=None)

## Prepare the stage
- Load spaCy

In [9]:
nlp = spacy.load('en_core_web_lg')

## Prepare the text
All the text handling and preparation concerned with the changes and modifications from the raw source text to a format that will be used for the actual processing, things like:
- handle encoding
- handle extraneous and international charaters
- handle simbols
- handle metadata and embeded information
- handle repetitions (such multiple spaces or newlines)

Clean text.

In [10]:
def clean_text(text):
    # reduce multiple spaces and newlines to only one
    text = re.sub(r'(\s\s+|\n\n+)', r'\1', text)
    # remove double quotes
    text = re.sub(r'"', '', text)

    return text

In [11]:
range(len(data_text.text))

range(0, 3000)

In [12]:
# ANSWER
for t in range(len(data_text.text)):
    data_text.text[t] = clean_text(data_text.text[t])

In [13]:
data_text.text

0                                Wow... Loved this place.
1                                      Crust is not good.
2               Not tasty and the texture was just nasty.
3       Stopped by during the late May bank holiday of...
4       The selection on the menu was great and so wer...
                              ...                        
2995    The screen does get smudged easily because it ...
2996    What a piece of junk.. I lose more calls on th...
2997                         Item Does Not Match Picture.
2998    The only thing that disappoint me is the infra...
2999    You can not answer calls with the unit, never ...
Name: text, Length: 3000, dtype: object

In [14]:
data_text.duplicated().sum()

17

In [15]:
data_text.drop_duplicates()

,text,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
2995,The screen does get smudged easily because it ...,0
2996,What a piece of junk.. I lose more calls on th...,0
2997,Item Does Not Match Picture.,0
2998,The only thing that disappoint me is the infra...,0


In [16]:
data_text.isnull().sum()

text     0
label    0
dtype: int64

## Work the text
Concern with the meaning and the substance of the content to extract actual information.

Hint: Use techniques learned in previous labs. Remove StopWords, Punctuation, Lemmatize etc.

In [17]:
!pip install spacytextblob

In [18]:
from spacy.lang.en.stop_words import STOP_WORDS
from spacytextblob.spacytextblob import SpacyTextBlob
nlp.add_pipe('spacytextblob')

In [19]:
def convert_text(text):
    '''
    Use techniques learned in previous labs. Remove StopWords, Punctuation, Lemmatize etc.
    '''
    print(text)
    # Removing stop words
    filtered_text = [] 
    filtered_lemma = [] 
    nlp_text = nlp(text)
    token_list = []
    for token in nlp_text:
        token_list.append(token.text)
        filtered_lemma.append(token.lemma_)
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            if lexeme.is_punct == False:
                filtered_text.append(word)
    #print(f"Token list: \n{token_list}")
    #print(f"\nFiltered text: \n{filtered_text}")
    #print(f"\nFiltered lemma: \n{filtered_lemma}")
    
    return filtered_text

In [20]:
def text_sentiment(text):
    
    # Get sentiment
    doc = nlp(text)
    print(f"polarity: {doc._.polarity}") 
    print(f"subjectivity: {doc._.subjectivity}")
    print(f"assessments: {doc._.assessments}")
    
    return doc._.polarity, doc._.subjectivity

In [21]:
convert_text(data_text.text[0])

Wow... Loved this place.


['Wow', 'Loved', 'place']

In [22]:
text_sentiment(data_text.text[0])

polarity: 0.39999999999999997
subjectivity: 0.9
assessments: [(['wow'], 0.1, 1.0, None), (['loved'], 0.7, 0.8, None)]


(0.39999999999999997, 0.9)

In [23]:
%%time
data_text['short'] = data_text['text'].apply(convert_text)

data_text['sentiment'] = data_text['text'].apply(text_sentiment)


Wow... Loved this place.
Crust is not good.
Not tasty and the texture was just nasty.
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.
The selection on the menu was great and so were the prices.
Now I am getting angry and I want my damn pho.
Honeslty it didn't taste THAT fresh.)
The potatoes were like rubber and you could tell they had been made up ahead of time being kept under a warmer.
The fries were great too.
A great touch.
Service was very prompt.
Would not go back.
The cashier had no care what so ever on what I had to say it still ended up being wayyy overpriced.
I tried the Cape Cod ravoli, chicken,with cranberry...mmmm!
I was disgusted because I was pretty sure that was human hair.
I was shocked because no signs indicate cash only.
Highly recommended.
Waitress was a little slow in service.
This place is not worth your time, let alone Vegas.
did not like at all.
The Burrittos Blah!
The food, amazing.
Service is also cute.
I could care less

Don't do it!!!!
The waiter wasn't helpful or friendly and rarely checked on us.
My husband and I ate lunch here and were very disappointed with the food and service.
And the red curry had so much bamboo shoots and wasn't very tasty to me.
Nice blanket of moz over top but i feel like this was done to cover up the subpar food.
The bathrooms are clean and the place itself is well decorated.
The menu is always changing, food quality is going down & service is extremely slow.
The service was a little slow , considering that were served by 3 people servers so the food was coming in a slow pace.
I give it 2 thumbs down
We watched our waiter pay a lot more attention to other tables and ignore us.
My fiancé and I came in the middle of the day and we were greeted and seated right away.
This is a great restaurant at the Mandalay Bay.
We waited for forty five minutes in vain.
Crostini that came with the salad was stale.
Some highlights : Great quality nigiri here!
the staff is friendly and the joi

I liked the patio and the service was outstanding.
The goat taco didn't skimp on the meat and wow what FLAVOR!
I think not again
I had the mac salad and it was pretty bland so I will not be getting that again.
I went to Bachi Burger on a friend's recommendation and was not disappointed.
Service stinks here!
I waited and waited.
This place is not quality sushi, it is not a quality restaurant.
I would definitely recommend the wings as well as the pizza.
Great Pizza and Salads!
Things that went wrong: - They burned the saganaki.
We waited an hour for what was a breakfast I could have done 100 times better at home.
This place is amazing!
I hate to disagree with my fellow Yelpers, but my husband and I were so disappointed with this place.
Waited 2 hours & never got either of our pizzas as many other around us who came in later did!
Just don't know why they were so slow.
The staff is great, the food is delish, and they have an incredible beer selection.
I live in the neighborhood so I am dis

I never come again.
We loved the biscuits!!!
Service is quick and friendly.
Ordered an appetizer and took 40 minutes and then the pizza another 10 minutes.
So absolutley fantastic.
It was a huge awkward 1.5lb piece of cow that was 3/4ths gristle and fat.
definitely will come back here again.
I like Steiners because it's dark and it feels like a bar.
Wow very spicy but delicious.
If you're not familiar, check it out.
I'll take my business dinner dollars elsewhere.
I'd love to go back.
Anyway, this FS restaurant has a wonderful breakfast/lunch.
Nothing special.
Each day of the week they have a different deal and it's all so delicious!
Not to mention the combination of pears, almonds and bacon is a big winner!
Will not be back.
Sauce was tasteless.
The food is delicious and just spicy enough, so be sure to ask for spicier if you prefer it that way.
My ribeye steak was cooked perfectly and had great mesquite flavor.
I don't think we'll be going back anytime soon.
Food was so gooodd.
I am f

After I pulled up my car I waited for another 15 minutes before being acknowledged.
Great food and great service in a clean and friendly setting.
All in all, I can assure you I'll be back.
I hate those things as much as cheap quality black olives.
My breakfast was perpared great, with a beautiful presentation of 3 giant slices of Toast, lightly dusted with powdered sugar.
The kids play area is NASTY!
Great place fo take out or eat in.
The waitress was friendly and happy to accomodate for vegan/veggie options.
OMG I felt like I had never eaten Thai food until this dish.
It was extremely crumby and pretty tasteless.
It was a pale color instead of nice and char and has NO flavor.
The croutons also taste homemade which is an extra plus.
I got home to see the driest damn wings ever!
It'll be a regular stop on my trips to Phoenix!
I really enjoyed Crema Café before they expanded; I even told friends they had the BEST breakfast.
Not good for the money.
I miss it and wish they had one in Phila

...

polarity: 0.5
subjectivity: 1.0
assessments: [(['glad'], 0.5, 1.0, None)]
polarity: 0.0
subjectivity: 0.5
assessments: [(['numerous'], 0.0, 0.5, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.5
subjectivity: 0.5
assessments: [(['most'], 0.5, 0.5, None)]
polarity: 0.3181818181818182
subjectivity: 0.4772727272727273
assessments: [(['more'], 0.5, 0.5, None), (['new'], 0.13636363636363635, 0.45454545454545453, None)]
polarity: -1.0
subjectivity: 1.0
assessments: [(['worst'], -1.0, 1.0, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.5
subjectivity: 0.78
assessments: [(['own'], 0.6, 1.0, None), (['very', 'happy'], 1.0, 1.0, None), (['sound'], 0.4, 0.4, None), (['especially'], 0.0, 1.0, None), (['better'], 0.5, 0.5, None)]
polarity: 0.48333333333333334
subjectivity: 0.7333333333333334
assessments: [(['love'], 0.5, 0.6, None), (['really', 'pretty'], 0.25, 1.0,

polarity: -0.8749999999999998
subjectivity: 0.6666666666666666
assessments: [(['bad', '!'], -0.8749999999999998, 0.6666666666666666, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.8
subjectivity: 0.75
assessments: [(['great'], 0.8, 0.75, None)]
polarity: 0.0
subjectivity: 0.55
assessments: [(['not', 'only'], -0.0, 1.0, None), (['extra'], 0.0, 0.1, None)]
polarity: 0.5
subjectivity: 1.0
assessments: [(['pleased'], 0.5, 1.0, None)]
polarity: -1.0
subjectivity: 1.0
assessments: [(['terrible'], -1.0, 1.0, None)]
polarity: 0.43333333333333335
subjectivity: 0.8333333333333334
assessments: [(['easy'], 0.43333333333333335, 0.8333333333333334, None)]
polarity: 0.21428571428571427
subjectivity: 0.42857142857142855
assessments: [(['large'], 0.21428571428571427, 0.42857142857142855, None)]
polarity: -0.35
subjectivity: 0.8
assessments: [(['utter'], 0.0, 1.0, None), (['crap'], -0.8, 0.8, 'profanity'), (['sound'], 0.4, 0.4, None), (['terrible'], -1.0, 1.0, None)]
polarity: 0.0
su

polarity: 0.2
subjectivity: 0.3
assessments: [(['very'], 0.2, 0.3, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.24047619047619048
subjectivity: 0.5160714285714286
assessments: [(['other'], -0.125, 0.375, None), (['nice'], 0.6, 1.0, None), (['soft'], 0.1, 0.35, None), (['fit'], 0.4, 0.4, None), (['very', 'tight'], -0.23214285714285715, 0.37142857142857144, None), (['good'], 0.7, 0.6000000000000001, None)]
polarity: 0.5
subjectivity: 0.5
assessments: [(['more'], 0.5, 0.5, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.8
subjectivity: 0.75
assessments: [(['great'], 0.8, 0.75, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 1.0
subjectivity: 0.3
assessments: [(['best'], 1.0, 0.3, None)]
polarity: -0.11111111111111112
subjectivity: 0.4222222222222222
assessments: [(['quickly'], 0.3333333333333333, 0.5, None), (['much', 'less'], -0.16666666666666666, 0.06666666666666667, None), (['expensive'], -0.5, 0.7, None)]
polarity: -0.5
subject

polarity: 0.5583333333333333
subjectivity: 0.55
assessments: [(['good'], 0.7, 0.6000000000000001, None), (['fine'], 0.4166666666666667, 0.5, None)]
polarity: 0.8
subjectivity: 0.75
assessments: [(['great'], 0.8, 0.75, None)]
polarity: 1.0
subjectivity: 0.75
assessments: [(['great', '!'], 1.0, 0.75, None)]
polarity: -0.5
subjectivity: 0.2
assessments: [(['completely', 'useless'], -0.5, 0.2, None)]
polarity: 0.625
subjectivity: 0.6
assessments: [(['love', '!'], 0.625, 0.6, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.475
subjectivity: 0.4666666666666667
assessments: [(['first'], 0.25, 0.3333333333333333, None), (['good'], 0.7, 0.6000000000000001, None)]
polarity: 1.0
subjectivity: 1.0
assessments: [(['excellent'], 1.0, 1.0, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.7
subjectivity: 0.6000000000000001
assessments: [(['good'], 0.7, 0.6000000000000001, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.4
subjectivity: 0.8
assessm

polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: -0.04999999999999999
subjectivity: 0.35
assessments: [(['sound'], 0.4, 0.4, None), (['failed'], -0.5, 0.3, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.15
subjectivity: 0.625
assessments: [(['ok'], 0.5, 0.5, None), (['not', 'great'], -0.4, 0.75, None), (['ok'], 0.5, 0.5, None), (['totally'], 0.0, 0.75, None)]
polarity: 0.0
subjectivity: 0.0
assessments: [(['back'], 0.0, 0.0, None)]
polarity: 0.0
subjectivity: 0.16666666666666666
assessments: [(['back'], 0.0, 0.0, None), (['enough'], 0.0, 0.5, None), (['next'], 0.0, 0.0, None)]
polarity: 0.125
subjectivity: 0.4
assessments: [(['wonderfully', 'wild', '!'], 0.125, 0.4, None)]
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.0
subjectivity: 0.0
assessments: []
polarity: 0.10416666666666666
subjectivity: 0.5208333333333333
assessments: [(['much', 'better'], 0.5, 0.5, None), (['hard'], -0.2916666666666667, 0.5416666666666666, None)]
polarity: 0.1363636

In [24]:
data_text.sample(10)

,text,label,short,sentiment
1865,"The characters are interesting, even if a bit ...",1,"[characters, interesting, bit, predictable, ]","(0.15, 0.5)"
1644,"Overall, a delight!",1,"[Overall, delight, ]","(0.0, 0.0)"
2608,"It is cheap, and it feel and look just as cheap.",0,"[cheap, feel, look, cheap]","(0.4, 0.7)"
483,The selection of food was not the best.,0,"[selection, food, best]","(1.0, 0.3)"
364,Server did a great job handling our large rowd...,1,"[Server, great, job, handling, large, rowdy, t...","(0.5071428571428571, 0.5892857142857143)"
1957,It showed exactly how the government and the s...,1,"[showed, exactly, government, scientist, argue...","(0.25, 0.25)"
2864,"But, in any case, the best part is, you can do...",1,"[case, best, download, pictures, laptop, IR, s...","(1.0, 0.3)"
266,"The service was not up to par, either.",0,"[service, par]","(0.0, 0.0)"
1780,The plot is nonsense that doesn't interest in ...,0,"[plot, nonsense, interest, slightest, way, uni...","(0.0, 0.0)"
1641,It's a campy sort of film that's a joy to watc...,1,"[campy, sort, film, joy, watch, ]","(0.8, 0.2)"


In [25]:
for t in range(len(data_text.short)):
    data_text.short[t] = ' '.join(data_text.short[t])

In [26]:
data_text.sample(10)

,text,label,short,sentiment
678,It's a great place and I highly recommend it.,1,great place highly recommend,"(0.48000000000000004, 0.645)"
2232,Don't waste your $$$ on this one.,0,waste $ $ $,"(-0.2, 0.0)"
2736,Krussel Always good.,1,Krussel good,"(0.7, 0.6000000000000001)"
1117,Then scene where they debated whether or not t...,0,scene debated sack trumpeter falsely accused m...,"(-0.32857142857142857, 0.7000000000000001)"
2127,it did not work in my cell phone plug i am ver...,0,work cell phone plug set charger,"(0.25, 0.3)"
2594,Better Than New.,1,Better New,"(0.3181818181818182, 0.4772727272727273)"
1620,"This is a masterful piece of film-making, with...",1,masterful piece film making themes simmering o...,"(0.5, 0.545)"
2347,Good audio quality.,1,Good audio quality,"(0.7, 0.6000000000000001)"
651,Great place to relax and have an awesome burge...,1,Great place relax awesome burger beer,"(0.9, 0.875)"
2434,But it does get better reception and clarity t...,1,better reception clarity phone,"(0.5, 0.5)"


## Modelling

In [27]:
# helper function to show results and charts
def show_summary_report(actual, prediction):

    if isinstance(actual, pd.Series):
        actual = actual.values
    if actual.dtype.name == 'object':
        actual = actual.astype(int)
    if prediction.dtype.name == 'object':
        prediction = prediction.astype(int)

    accuracy_ = accuracy_score(actual, prediction)
    precision_ = precision_score(actual, prediction)
    recall_ = recall_score(actual, prediction)
    roc_auc_ = roc_auc_score(actual, prediction)

    print('Accuracy : %.4f [TP / N] Proportion of predicted labels that match the true labels. Best: 1, Worst: 0' % accuracy_)
    print('Precision: %.4f [TP / (TP + FP)] Not to label a negative sample as positive.        Best: 1, Worst: 0' % precision_)
    print('Recall   : %.4f [TP / (TP + FN)] Find all the positive samples.                     Best: 1, Worst: 0' % recall_)
    print('ROC AUC  : %.4f                                                                     Best: 1, Worst: < 0.5' % roc_auc_)
    print('-' * 107)
    print('TP: True Positives, FP: False Positives, TN: True Negatives, FN: False Negatives, N: Number of samples')

    # Confusion Matrix
    mat = confusion_matrix(actual, prediction)

    # Precision/Recall
    precision, recall, _ = precision_recall_curve(actual, prediction)
    average_precision = average_precision_score(actual, prediction)
    
    # Compute ROC curve and ROC area
    fpr, tpr, _ = roc_curve(actual, prediction)
    roc_auc = auc(fpr, tpr)

    # plot
    fig, ax = plt.subplots(1, 3, figsize = (18, 6))
    fig.subplots_adjust(left = 0.02, right = 0.98, wspace = 0.2)

    # Confusion Matrix
    sns.heatmap(mat.T, square = True, annot = True, fmt = 'd', cbar = False, cmap = 'Blues', ax = ax[0])

    ax[0].set_title('Confusion Matrix')
    ax[0].set_xlabel('True label')
    ax[0].set_ylabel('Predicted label')
    
    # Precision/Recall
    step_kwargs = {'step': 'post'}
    ax[1].step(recall, precision, color = 'b', alpha = 0.2, where = 'post')
    ax[1].fill_between(recall, precision, alpha = 0.2, color = 'b', **step_kwargs)
    ax[1].set_ylim([0.0, 1.0])
    ax[1].set_xlim([0.0, 1.0])
    ax[1].set_xlabel('Recall')
    ax[1].set_ylabel('Precision')
    ax[1].set_title('2-class Precision-Recall curve')

    # ROC
    ax[2].plot(fpr, tpr, color = 'darkorange', lw = 2, label = 'ROC curve (AUC = %0.2f)' % roc_auc)
    ax[2].plot([0, 1], [0, 1], color = 'navy', lw = 2, linestyle = '--')
    ax[2].set_xlim([0.0, 1.0])
    ax[2].set_ylim([0.0, 1.0])
    ax[2].set_xlabel('False Positive Rate')
    ax[2].set_ylabel('True Positive Rate')
    ax[2].set_title('Receiver Operating Characteristic')
    ax[2].legend(loc = 'lower right')

    plt.show()
    
    return (accuracy_, precision_, recall_, roc_auc_)

In [44]:
# Features and Labels
X = data_text['short']
y = data_text['sentiment']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Use Bag-of-Words

In [45]:
# create a matrix of word counts from the text
counts = CountVectorizer()

In [46]:
# do the actual counting
A = counts.fit_transform(X_train, y_train)

In [47]:
pd.DataFrame(A.todense(), columns=counts.get_feature_names()).head()

,00,10,100,11,12,13,15,15pm,17,18,...,yucky,yukon,yum,yummy,yun,z500a,zero,zillion,zombie,zombiez
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# create a classifier using SVC
classifier = SVC(kernel='linear', probability=True)

In [ ]:
# train the classifier with the training data
classifier.fit(A.toarray(), y_train)

In [50]:
# do the transformation for the test data
# NOTE: use `transform()` instead of `fit_transform()`
B = counts.transform(X_test)

In [ ]:
# make predictions based on the test data
predictions = classifier.predict(B)
predictions

In [ ]:
# check the accuracy
print('Accuracy: %.4f' % accuracy_score(y_test, predictions))

In [ ]:
show_summary_report(y_test, predictions, probabilities)

## Repeat using TF-IDF
TF-IDF = Term Frequency - Inverse Document Frequency

In [ ]:
# create a matrix of word counts from the text
# use TF-IDF
tfidf = TfidfVectorizer()
# do the actual counting
A = tfidf.fit_transform(X_train, y_train)

# train the classifier with the training data
classifier.fit(A.toarray(), y_train)

# do the transformation for the test data
# NOTE: use `transform()` instead of `fit_transform()`
B = tfidf.transform(X_test)

# make predictions based on the test data
predictions = classifier.predict(B)

# check the accuracy
print('Accuracy: %.4f' % accuracy_score(y_test, predictions))

## Repeating it all for comparision
Repeating the whole lot in one big block

Find 'Accuracy', 'Precision', 'Recall', 'ROC_AUC' using CountVectorizer and TfidfVectorizer and keep the result in a dataframe.

In [ ]:
# ANSWER

# Keep the results in a dataframe
results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'ROC_AUC'])

models = ['Count', 'TF-IDX']
for i, m in enumerate([CountVectorizer(), TfidfVectorizer()]):
    print('*' * (len(models[i]) + 4))
    print('* %s *' % models[i])
    print('*' * (len(models[i]) + 4))
    
    # create a matrix of word counts from the text
    counts = m
    # do the actual counting
    A = counts.fit_transform(X_train, y_train)
    
    # create a classifier using SVC
    classifier = SVC(kernel='linear', probability=True)
    
    # train the classifier with the training data
    classifier.fit(A.toarray(), y_train)
    
    # do the transformation for the test data
    # NOTE: use `transform()` instead of `fit_transform()`
    B =counts.transform(X_test)
    
    # make predictions based on the test data
    predictions =classifier.predict(B.todense())
    
    # store probabilities of predictions being 1
    probabilities = classifier.predict_proba(B.todense())[:, 1]
    
    # show the report
    accuracy_, precision_, recall_, roc_auc_ = show_summary_report(y_test, predictions, probabilities)
    # keep the results
    results.loc[i] = {'Model': models[i], 
                      'Accuracy': accuracy_, 
                      'Precision': precision_,
                      'Recall': recall_,
                      'ROC_AUC': roc_auc_}
    
    
    



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



